In [2]:
# For Google Colaboratory
import sys, os
if 'google.colab' in sys.modules:
    # mount google drive
    from google.colab import drive
    drive.mount('/content/gdrive')
    # find automatically the path of the folder containing "file_name" :
    file_name = 'n3-rest.ipynb'
    import subprocess
    path_to_file = subprocess.check_output('find . -type f -name ' + str(file_name), shell=True).decode("utf-8")
    path_to_file = path_to_file.replace(file_name,"").replace('\n',"")
    # if previous search failed or too long, comment the previous line and simply write down manually the path below :
    #path_to_file = '/content/gdrive/My Drive/AI6103_2020_codes/codes/labs_lecture08/lab04_vgg'
    print(path_to_file)
    # change current path to the folder containing "file_name"
    os.chdir(path_to_file)
    !pwd

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.metrics.pairwise import cosine_similarity
%matplotlib inline

In [4]:
file_name = "11.csv"
df = pd.read_csv(file_name, sep=',')
df.head(5).append(df.tail(5))

,number,title,genre,plot,Running time,Box office,imdb
0,1,Snow White and the Seven Dwarfs,"Animation, Family, Fantasy, Musical, Romance",Exiled into the dangerous forest by her wicked...,83,4.180000e+08,7.6
1,2,Pinocchio,"Animation, Comedy, Family, Fantasy, Musical","A living puppet, with the help of a cricket as...",88,1.640000e+08,7.4
2,3,Fantasia,"Animation, Family, Fantasy, Music",A collection of animated interpretations of gr...,126,8.330000e+07,7.7
3,4,The Reluctant Dragon,"Animation, Comedy, Family",Humorist Robert Benchley learns about the anim...,74,9.600000e+05,6.9
4,5,Dumbo,"Adventure, Family, Fantasy","A young elephant, whose oversized ears enable ...",64,1.300000e+06,7.2
234,235,Penguins,"Documentary, Family",A coming-of-age story about an Adélie penguin ...,76,7.700000e+00,7.0
235,236,Toy Story 4,"Animation, Adventure, Comedy, Family, Fantasy","When a new toy called ""Forky"" joins Woody and ...",100,1.073000e+09,7.8
236,237,The Lion King,"Animation, Adventure, Drama, Family, Musical","After the murder of his father, a young lion p...",118,1.657000e+09,8.5
237,238,Frozen II,"Animation, Adventure, Comedy, Family, Fantasy,...","Anna, Elsa, Kristoff, Olaf and Sven leave Aren...",103,1.450000e+09,6.9
238,239,Onward,"Animation, Adventure, Comedy, Family, Fantasy",Two elven brothers embark on a quest to bring ...,102,1.419000e+08,7.4


In [5]:
display(df.shape)
display(df.info())

(239, 7)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 239 entries, 0 to 238
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   number        239 non-null    int64  
 1   title         239 non-null    object 
 2   genre         239 non-null    object 
 3   plot          239 non-null    object 
 4   Running time  239 non-null    int64  
 5   Box office    239 non-null    float64
 6   imdb          235 non-null    float64
dtypes: float64(2), int64(2), object(3)
memory usage: 13.2+ KB


None

In [6]:
df.describe()

,number,Running time,Box office,imdb
count,239.000000,239.000000,2.390000e+02,235.000000
mean,120.000000,95.974895,1.945666e+08,6.611915
std,69.137544,19.228376,2.955427e+08,1.030821
min,1.000000,40.000000,7.700000e+00,1.500000
25%,60.500000,83.000000,1.820000e+07,6.100000
50%,120.000000,95.000000,5.850000e+07,6.800000
75%,179.500000,106.000000,2.515500e+08,7.250000
max,239.000000,168.000000,1.657000e+09,8.500000


In [7]:
df['original_title'] = df['title']
df['imdb'] = df['imdb'].fillna(df['imdb'].mean())
df['Running time'] = df['Running time'].fillna(df['Running time'].mean())

min_max_scaler = preprocessing.MinMaxScaler()
df['norm_running_time'] = min_max_scaler.fit_transform(df['Running time'].values.reshape(-1, 1))
df['norm_imdb'] = min_max_scaler.fit_transform(df['imdb'].values.reshape(-1, 1))


In [8]:
df.drop(columns=['Running time', 'Box office', 'imdb'], inplace=True)
df.head()

,number,title,genre,plot,original_title,norm_running_time,norm_imdb
0,1,Snow White and the Seven Dwarfs,"Animation, Family, Fantasy, Musical, Romance",Exiled into the dangerous forest by her wicked...,Snow White and the Seven Dwarfs,0.335938,0.871429
1,2,Pinocchio,"Animation, Comedy, Family, Fantasy, Musical","A living puppet, with the help of a cricket as...",Pinocchio,0.375000,0.842857
2,3,Fantasia,"Animation, Family, Fantasy, Music",A collection of animated interpretations of gr...,Fantasia,0.671875,0.885714
3,4,The Reluctant Dragon,"Animation, Comedy, Family",Humorist Robert Benchley learns about the anim...,The Reluctant Dragon,0.265625,0.771429
4,5,Dumbo,"Adventure, Family, Fantasy","A young elephant, whose oversized ears enable ...",Dumbo,0.187500,0.814286


In [9]:
import nltk
nltk.download('brown')
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\生菜\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\生菜\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\生菜\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
from nltk.corpus import brown
brown.words()
['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', ...]

['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', Ellipsis]

In [11]:
from nltk.tokenize import word_tokenize

df['title'] = df['title'].str.lower()
df['genre'] = df['genre'].str.lower()
df['plot'] = df['plot'].str.lower()

df['title'] = df['title'].apply(word_tokenize)
df['genre'] = df['genre'].apply(word_tokenize)
df['plot'] = df['plot'].apply(word_tokenize)

In [12]:
from nltk.corpus import stopwords
from string import punctuation

list_stopwords = set(stopwords.words('english') + list(punctuation))
df['title'] = df['title'].apply(lambda x: [word for word in x if word not in list_stopwords])
df['genre'] = df['genre'].apply(lambda x: [word for word in x if word not in list_stopwords])
df['plot'] = df['plot'].apply(lambda x: [word for word in x if word not in list_stopwords])

In [13]:
import string

df['plot'] = df['plot'].apply(lambda x : [word.translate(str.maketrans('', '', string.punctuation)) for word in x])
df['plot'] = df['plot'].apply(lambda x : [word for word in x if len(word) > 0])

In [14]:
df['title'] = df['title'].apply(lambda x : list(set(x)))
df['genre'] = df['genre'].apply(lambda x : list(set(x)))
df['plot'] = df['plot'].apply(lambda x : list(set(x)))
df['title']

0      [white, snow, seven, dwarfs]
1                       [pinocchio]
2                        [fantasia]
3               [reluctant, dragon]
4                           [dumbo]
                   ...             
234                      [penguins]
235                 [4, story, toy]
236                    [lion, king]
237                    [ii, frozen]
238                        [onward]
Name: title, Length: 239, dtype: object

In [15]:
df

,number,title,genre,plot,original_title,norm_running_time,norm_imdb
0,1,"[white, snow, seven, dwarfs]","[animation, musical, romance, family, fantasy]","[wicked, part, dwarf, princess, dangerous, sev...",Snow White and the Seven Dwarfs,0.335938,0.871429
1,2,[pinocchio],"[animation, musical, comedy, family, fantasy]","[living, puppet, become, help, must, boy, real...",Pinocchio,0.375000,0.842857
2,3,[fantasia],"[fantasy, animation, family, music]","[collection, interpretations, animated, great,...",Fantasia,0.671875,0.885714
3,4,"[reluctant, dragon]","[animation, family, comedy]","[find, making, walt, animation, cartoon, idea,...",The Reluctant Dragon,0.265625,0.771429
4,5,[dumbo],"[fantasy, adventure, family]","[shiny, whose, enable, circus, veneer, helps, ...",Dumbo,0.187500,0.814286
...,...,...,...,...,...,...,...
234,235,[penguins],"[documentary, family]","[spring, icy, nest, find, steve, adélie, life,...",Penguins,0.281250,0.785714
235,236,"[4, story, toy]","[adventure, animation, comedy, family, fantasy]","[new, old, gang, big, trip, road, called, reve...",Toy Story 4,0.468750,0.900000
236,237,"[lion, king]","[adventure, animation, musical, family, drama]","[true, prince, meaning, murder, bravery, flees...",The Lion King,0.609375,1.000000
237,238,"[ii, frozen]","[adventure, animation, musical, comedy, family...","[ancient, enchanted, find, sven, leave, elsa, ...",Frozen II,0.492188,0.771429


In [16]:
!pip install gensim==3.8.3

In [17]:
import gensim

wv = gensim.models.KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin.gz", binary=True)

In [18]:
matrix_key_to_index = []
for list_ in df.to_numpy():
    list_[2] = [word for word in list_[2] if word in wv.vocab]
    list_[3] = [word for word in list_[3] if word in wv.vocab]
    list_[1] = [word for word in list_[1] if word in wv.vocab]
    matrix_key_to_index.append(list_)
df_key_to_index = pd.DataFrame(matrix_key_to_index, columns=df.columns)
df_key_to_index

,number,title,genre,plot,original_title,norm_running_time,norm_imdb
0,1,"[white, snow, seven, dwarfs]","[animation, musical, romance, family, fantasy]","[wicked, part, dwarf, princess, dangerous, sev...",Snow White and the Seven Dwarfs,0.335938,0.871429
1,2,[],"[animation, musical, comedy, family, fantasy]","[living, puppet, become, help, must, boy, real...",Pinocchio,0.375000,0.842857
2,3,[fantasia],"[fantasy, animation, family, music]","[collection, interpretations, animated, great,...",Fantasia,0.671875,0.885714
3,4,"[reluctant, dragon]","[animation, family, comedy]","[find, making, walt, animation, cartoon, idea,...",The Reluctant Dragon,0.265625,0.771429
4,5,[dumbo],"[fantasy, adventure, family]","[shiny, whose, enable, circus, veneer, helps, ...",Dumbo,0.187500,0.814286
...,...,...,...,...,...,...,...
234,235,[penguins],"[documentary, family]","[spring, icy, nest, find, steve, life, fellow,...",Penguins,0.281250,0.785714
235,236,"[4, story, toy]","[adventure, animation, comedy, family, fantasy]","[new, old, gang, big, trip, road, called, reve...",Toy Story 4,0.468750,0.900000
236,237,"[lion, king]","[adventure, animation, musical, family, drama]","[true, prince, meaning, murder, bravery, flees...",The Lion King,0.609375,1.000000
237,238,"[ii, frozen]","[adventure, animation, musical, comedy, family...","[ancient, enchanted, find, sven, leave, anna, ...",Frozen II,0.492188,0.771429


In [19]:
from tqdm import tqdm
    
def recommendation(title, title_weight=0.25, genre_weight=0.25, plot_weight=0.25, half_factor=None):
    matrix_title_key_to_index = []
    for list_ in df[df['original_title'] == title].to_numpy():
        list_[2] = [word for word in list_[2] if word in wv.vocab]
        list_[3] = [word for word in list_[3] if word in wv.vocab]
        list_[1] = [word for word in list_[1] if word in wv.vocab]
        matrix_title_key_to_index.append(list_)
    
    matrix_similarity = []
    pbar = tqdm(matrix_key_to_index)
    for list1 in pbar:
        for list2 in matrix_title_key_to_index:
            score_genre = wv.n_similarity(list1[2], list2[2])
            score_plot = wv.n_similarity(list1[3], list2[3])
            try:
                score_title = wv.n_similarity(list1[1], list2[1])
            except:
                score_title = 0
            score_time_imdb = cosine_similarity([[list1[5], list1[6]]], [[list2[5], list2[6]]])
        if ((list1[1] != list2[1]) & (list1[5] != list2[5])):
                matrix_similarity.append([list1[4], list2[4], score_title, score_genre, score_plot, score_time_imdb[0][0]])
        pbar.update()
    pbar.close()
    if half_factor == 'title':
        title_weight = 0.5
        genre_weight = 0.25
        plot_weight = 0.25
    elif half_factor == 'genre':
        title_weight = 0.25
        genre_weight = 0.5
        plot_weight = 0.25
    elif half_factor == 'plot':
        title_weight = 0.25
        genre_weight = 0.25
        plot_weight = 0.5
    else:
        plot_weight = 1 - title_weight - genre_weight
    time_imdb_weight = 1 - title_weight - genre_weight - plot_weight
    df_similarity = pd.DataFrame(matrix_similarity, columns = ['recommendation','title','score_title', 'score_genre', 'score_plot', 'score_time_imdb'])
    df_similarity['final_score'] = df_similarity['score_title'] * title_weight + df_similarity['score_genre'] * genre_weight + df_similarity['score_plot'] * plot_weight + df_similarity['score_time_imdb'] * time_imdb_weight
    return (df_similarity.sort_values(by=['final_score', 'score_genre', 'score_plot', 'score_title', 'score_time_imdb'], ascending=False).head(10))

In [20]:
recommendation('Frozen', title_weight=0.3, genre_weight=0.3, plot_weight=0.3)

100%|███████████████████████████████████████████████████████████████████████████████| 239/239 [00:01<00:00, 180.90it/s]


,recommendation,title,score_title,score_genre,score_plot,score_time_imdb,final_score
236,Frozen II,Frozen,0.687350,1.000000,0.596618,0.998928,0.785083
0,Snow White and the Seven Dwarfs,Frozen,0.264768,0.925796,0.679161,0.988219,0.659739
13,Alice in Wonderland,Frozen,0.159988,0.967738,0.685363,0.988891,0.642816
223,Finding Dory,Frozen,0.194784,0.938075,0.666480,0.999596,0.639761
73,"Honey, I Shrunk the Kids",Frozen,0.290419,0.859469,0.634851,0.999921,0.635414
166,Chicken Little,Frozen,0.330215,0.943610,0.509357,0.999512,0.634906
193,The Princess and the Frog,Frozen,0.162635,0.977926,0.642305,0.999907,0.634851
104,James and the Giant Peach,Frozen,0.149947,0.967738,0.661946,0.991253,0.633015
134,Snow Dogs,Frozen,0.275487,0.826855,0.673252,0.980939,0.630772
142,Piglet's Big Movie,Frozen,0.187037,0.937290,0.638985,0.990921,0.628086


In [21]:
df

,number,title,genre,plot,original_title,norm_running_time,norm_imdb
0,1,"[white, snow, seven, dwarfs]","[animation, musical, romance, family, fantasy]","[wicked, part, dwarf, princess, dangerous, sev...",Snow White and the Seven Dwarfs,0.335938,0.871429
1,2,[pinocchio],"[animation, musical, comedy, family, fantasy]","[living, puppet, become, help, must, boy, real...",Pinocchio,0.375000,0.842857
2,3,[fantasia],"[fantasy, animation, family, music]","[collection, interpretations, animated, great,...",Fantasia,0.671875,0.885714
3,4,"[reluctant, dragon]","[animation, family, comedy]","[find, making, walt, animation, cartoon, idea,...",The Reluctant Dragon,0.265625,0.771429
4,5,[dumbo],"[fantasy, adventure, family]","[shiny, whose, enable, circus, veneer, helps, ...",Dumbo,0.187500,0.814286
...,...,...,...,...,...,...,...
234,235,[penguins],"[documentary, family]","[spring, icy, nest, find, steve, adélie, life,...",Penguins,0.281250,0.785714
235,236,"[4, story, toy]","[adventure, animation, comedy, family, fantasy]","[new, old, gang, big, trip, road, called, reve...",Toy Story 4,0.468750,0.900000
236,237,"[lion, king]","[adventure, animation, musical, family, drama]","[true, prince, meaning, murder, bravery, flees...",The Lion King,0.609375,1.000000
237,238,"[ii, frozen]","[adventure, animation, musical, comedy, family...","[ancient, enchanted, find, sven, leave, elsa, ...",Frozen II,0.492188,0.771429


In [22]:
def CalculateFrequency():
    """统计plot中的各个单词的频率"""
    Frequency_plot = {}
    for plot in df["plot"]:
        for word in plot:
            if word not in Frequency_plot:
                Frequency_plot[word] = 1
            else:
                Frequency_plot[word] += 1
    plot_rank = sorted(Frequency_plot,key=lambda x:Frequency_plot[x],reverse=True)
    return [(word,Frequency_plot[word]) for word in plot_rank]

In [23]:
Frequencies = CalculateFrequency()

In [24]:
Frequencies[:20]

[('s', 42),
 ('young', 30),
 ('find', 29),
 ('boy', 23),
 ('new', 23),
 ('family', 21),
 ('friends', 18),
 ('home', 17),
 ('world', 17),
 ('life', 17),
 ('must', 16),
 ('two', 16),
 ('man', 15),
 ('help', 14),
 ('father', 14),
 ('save', 13),
 ('back', 13),
 ('team', 12),
 ('true', 12),
 ('set', 11)]

In [25]:
def SearchMovies(keyword):
    matchedMovies = pd.DataFrame()
    matchedMovies_titles = []
    matchedMovies_plots = []
    original_titles = []
    for i in range(len(df["plot"])):
        if keyword in df["plot"][i]:
            matchedMovies_titles.append(df["title"][i])
            matchedMovies_plots.append(df["plot"][i])
            original_titles.append(df["original_title"][i])

    matchedMovies["original_title"] = original_titles
    matchedMovies["title"] = matchedMovies_titles
    matchedMovies["plot"] = matchedMovies_plots
    return matchedMovies

In [26]:
SearchMovies("life")

,original_title,title,plot
0,Pollyanna,[pollyanna],"[girl, embittered, town, comes, see, young, de..."
1,Jungle Cat,"[jungle, cat]","[documentary, female, jaguar, south, american,..."
2,Swiss Family Robinson,"[robinson, family, swiss]","[home, new, environment, deserted, shipwrecked..."
3,First Kid,"[first, kid]","[boxer, service, agent, son, difficult, new, p..."
4,Jungle 2 Jungle,"[2, jungle]","[son, york, new, man, boy, thirteenyearold, ju..."
5,The Kid,[kid],"[eightyearold, unhappy, consultant, confronted..."
6,Freaky Friday,"[friday, freaky]","[one, adapt, get, overworked, along, friday, l..."
7,The Incredibles,[incredibles],"[save, action, undercover, life, quiet, suburb..."
8,Aliens of the Deep,"[aliens, deep]","[home, chain, ridge, explore, james, midocean,..."
9,Bedtime Stories,"[bedtime, stories]","[lavish, true, bedtime, tells, magically, hote..."


In [27]:
idx = df["original_title"].tolist().index("Zootopia")
df["title"][idx] = ["zoo","animal"]

<ipython-input-27-048e7f82498c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["title"][idx] = ["zoo","animal"]


In [31]:
idx = df["original_title"].tolist().index("Pinocchio")
df["title"][idx] = ["puppet","nose"]

<ipython-input-31-3a7a59f8cfad>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["title"][idx] = ["puppet","nose"]


In [32]:
recommendation('Pinocchio')

100%|███████████████████████████████████████████████████████████████████████████████| 239/239 [00:00<00:00, 631.80it/s]


,recommendation,title,score_title,score_genre,score_plot,score_time_imdb,final_score
1,Pinocchio,Pinocchio,0.000000,1.000000,1.000000,1.000000,0.750000
34,The Sword in the Stone,Pinocchio,0.303418,0.971244,0.651795,0.998166,0.731156
193,The Princess and the Frog,Pinocchio,0.394509,0.941603,0.589546,0.996015,0.730418
110,Hercules,Pinocchio,0.245154,0.941603,0.654739,0.998996,0.710123
205,The Muppets,Pinocchio,0.344137,0.865639,0.636482,0.991178,0.709359
96,A Goofy Movie,Pinocchio,0.298608,0.911009,0.622176,0.998687,0.707620
161,Pooh's Heffalump Movie,Pinocchio,0.376873,0.958977,0.495736,0.993310,0.706224
3,The Reluctant Dragon,Pinocchio,0.338641,0.902703,0.587030,0.996218,0.706148
100,Toy Story,Pinocchio,0.300669,0.928607,0.590090,0.994993,0.703590
105,The Hunchback of Notre Dame,Pinocchio,0.302420,0.874214,0.636929,0.998310,0.702968


In [29]:
recommendation('Zootopia', title_weight=0.3, genre_weight=0.3, plot_weight=0.3)

100%|███████████████████████████████████████████████████████████████████████████████| 239/239 [00:00<00:00, 674.32it/s]


,recommendation,title,score_title,score_genre,score_plot,score_time_imdb,final_score
107,101 Dalmatians,Zootopia,0.421024,0.912554,0.702221,0.986031,0.709343
222,Zootopia,Zootopia,0.000000,1.000000,1.000000,1.000000,0.700000
108,That Darn Cat,Zootopia,0.432209,0.885830,0.674186,0.989279,0.696595
151,Teacher's Pet,Zootopia,0.492506,0.851876,0.640481,0.980248,0.693483
74,Cheetah,Zootopia,0.565534,0.700705,0.633229,0.999289,0.669769
142,Piglet's Big Movie,Zootopia,0.399961,0.925629,0.542076,0.991178,0.659418
172,The Wild,Zootopia,0.393048,0.898058,0.565291,0.999712,0.656890
193,The Princess and the Frog,Zootopia,0.397754,0.854898,0.603110,0.999931,0.656722
220,The Good Dinosaur,Zootopia,0.371112,0.878208,0.585926,0.999995,0.650573
128,102 Dalmatians,Zootopia,0.421024,0.837541,0.580415,0.969414,0.648635


In [30]:
idx = df["original_title"].tolist().index("Zootopia")
df["title"][idx] = ["zootopia"]
recommendation('Zootopia', title_weight=0.3, genre_weight=0.3, plot_weight=0.3)

<ipython-input-30-8a316f56d5b8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["title"][idx] = ["zootopia"]
100%|███████████████████████████████████████████████████████████████████████████████| 239/239 [00:00<00:00, 741.44it/s]


,recommendation,title,score_title,score_genre,score_plot,score_time_imdb,final_score
101,101 Dalmatians,Zootopia,0,0.912554,0.702221,0.986031,0.583036
203,Muppets Most Wanted,Zootopia,0,0.935037,0.656515,0.992518,0.576718
109,A Bug's Life,Zootopia,0,0.898058,0.662446,0.999418,0.568093
102,That Darn Cat,Zootopia,0,0.885830,0.674186,0.989279,0.566933
27,One Hundred and One Dalmatians,Zootopia,0,0.898058,0.651298,0.986040,0.563411
2,The Reluctant Dragon,Zootopia,0,0.836292,0.710064,0.982371,0.562144
45,Robin Hood,Zootopia,0,0.886375,0.653743,0.996163,0.561652
72,The Rescuers Down Under,Zootopia,0,0.935268,0.601728,0.987044,0.559803
115,Toy Story 2,Zootopia,0,0.874538,0.646554,0.994850,0.555813
129,Lilo & Stitch,Zootopia,0,0.866661,0.654440,0.993724,0.555702
